# EXTRACT EVENTS DARTMOUTH FLOOD OBSERVATORY

The file used can be found at: http://floodobservatory.colorado.edu/Archives/index.html

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
import re
import io
import csv
import os

In [3]:
INPUT_FILE = "./data/Dartmouth/FloodArchive.xlsx" #Download from link above

data = pd.read_excel(INPUT_FILE, index_col=0)

In [4]:
data.head()

,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity
ID,,,,,,,,,,,,,
1,0,Algeria,0,5.23026,35.81420,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0
2,0,Brazil,0,-45.34890,-18.71110,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0
3,0,Phillipines,0,122.97400,10.02070,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0
4,0,Indonesia,0,124.60600,1.01489,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0
5,0,Mozambique,0,32.34910,-25.86930,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0


In [5]:
data.index = data.pop('Began')

In [6]:
data = data.loc['2018-08-01':'2019-04-01'] #We select events within the range we are interested in

In [7]:
data.head()

,GlideNumber,Country,OtherCountry,long,lat,Area,Ended,Validation,Dead,Displaced,MainCause,Severity
Began,,,,,,,,,,,,
2018-08-09,NaN,France,NaN,5.78789,44.41810,56965.28,2018-08-10,FloodList,0,1600,Torrential Rain,1.5
2018-08-02,NaN,Algeria,NaN,6.40648,22.97200,349302.36,2018-08-09,FloodList,5,0,Heavy Rain,1.5
2018-08-07,NaN,Canada,NaN,-78.92150,43.94050,4707.54,2018-08-09,FloodList,0,0,Torrential Rain,1.0
2018-08-07,NaN,India,NaN,76.72750,9.39943,24440.47,2018-08-10,Flood List,20,1000,Monsoonal Rain,1.5
2018-08-07,NaN,Turkey,NaN,40.52110,40.93070,23112.35,2018-08-10,FloodList,0,300,Heavy Rain,1.5


In [8]:
data = data.drop(columns=['GlideNumber', 'Validation', 'Severity'])
data.rename(columns={'Country': 'countries', 'Dead': 'deaths', 'Displaced': 'affected', 'Ended': 'end_date', 'MainCause': 'cause', 'ID': 'id_darmouth', 'Area': 'area'}, inplace=True)
data.rename_axis('start_date', inplace=True)

In [9]:
data = data.sort_values(by='start_date')
data['in_darmouth'] = 1 #Add identifier of source for future merging process
data.head()

,countries,OtherCountry,long,lat,area,end_date,deaths,affected,cause
start_date,,,,,,,,,
2018-08-01,Sudan,NaN,31.94180,13.48700,629241.72,2018-09-04,23,4500,Heavy Rain
2018-08-02,Algeria,NaN,6.40648,22.97200,349302.36,2018-08-09,5,0,Heavy Rain
2018-08-02,India,NaN,94.12220,27.29140,73328.62,2018-08-10,0,25000,Monsoonal Rain
2018-08-07,Canada,NaN,-78.92150,43.94050,4707.54,2018-08-09,0,0,Torrential Rain
2018-08-07,India,NaN,76.72750,9.39943,24440.47,2018-08-10,20,1000,Monsoonal Rain
...,...,...,...,...,...,...,...,...,...
2019-03-17,Iran,Afghanistan,61.58450,30.13460,986684.27,2019-04-02,101,56000,Heavy Rain and Torrential Rain
2019-03-23,Burundi,NaN,29.88810,-3.14741,9362.24,2019-03-29,0,25,Torrential Rain
2019-03-25,New Zealand,NaN,171.29700,-42.76380,36524.78,2019-03-27,1,0,Torrential Rain


In [10]:
data.reset_index(inplace = True)

for index, row in data.iterrows(): #We join several countries in one single column
    if (not pd.isnull(row['OtherCountry'])):
        data.at[index, 'countries'] = (", ".join([row['countries'], row['OtherCountry']]))

Myanmar
Oman
Slovenia
Iran
Israel
Malaysia
Malawi
Peru
Mozambique
Pakistan
Jordan
Bolivia
Afghanistan


,start_date,id_darmouth,countries,OtherCountry,long,lat,area,end_date,deaths,affected,cause,in_darmouth
0,2018-08-01,4668,Sudan,NaN,31.94180,13.48700,629241.72,2018-09-04,23,4500,Heavy Rain,1
1,2018-08-02,4660,Algeria,NaN,6.40648,22.97200,349302.36,2018-08-09,5,0,Heavy Rain,1
2,2018-08-02,4665,India,NaN,94.12220,27.29140,73328.62,2018-08-10,0,25000,Monsoonal Rain,1
3,2018-08-07,4661,Canada,NaN,-78.92150,43.94050,4707.54,2018-08-09,0,0,Torrential Rain,1
4,2018-08-07,4663,India,NaN,76.72750,9.39943,24440.47,2018-08-10,20,1000,Monsoonal Rain,1
5,2018-08-07,4664,Turkey,NaN,40.52110,40.93070,23112.35,2018-08-10,0,300,Heavy Rain,1
6,2018-08-09,4659,France,NaN,5.78789,44.41810,56965.28,2018-08-10,0,1600,Torrential Rain,1
7,2018-08-15,4675,"India, Myanmar",Myanmar,94.76550,26.05130,28864.54,2018-09-07,11,2000,Monsoonal Rain,1
8,2018-08-20,4667,Nigeria,NaN,8.23750,10.47480,257477.37,2018-10-02,108,148000,Heavy Rain,1
9,2018-08-22,4670,USA,NaN,-155.44100,19.66890,7451.62,2018-08-31,0,50,Tropical Storm Lane,1


In [11]:
data = data.drop(columns=['OtherCountry'])
data.index = data.pop('start_date')
data.head()

,start_date,id_darmouth,countries,long,lat,area,end_date,deaths,affected,cause,in_darmouth
0,2018-08-01,4668,Sudan,31.94180,13.48700,629241.72,2018-09-04,23,4500,Heavy Rain,1
1,2018-08-02,4660,Algeria,6.40648,22.97200,349302.36,2018-08-09,5,0,Heavy Rain,1
2,2018-08-02,4665,India,94.12220,27.29140,73328.62,2018-08-10,0,25000,Monsoonal Rain,1
3,2018-08-07,4661,Canada,-78.92150,43.94050,4707.54,2018-08-09,0,0,Torrential Rain,1
4,2018-08-07,4663,India,76.72750,9.39943,24440.47,2018-08-10,20,1000,Monsoonal Rain,1
5,2018-08-07,4664,Turkey,40.52110,40.93070,23112.35,2018-08-10,0,300,Heavy Rain,1
6,2018-08-09,4659,France,5.78789,44.41810,56965.28,2018-08-10,0,1600,Torrential Rain,1
7,2018-08-15,4675,"India, Myanmar",94.76550,26.05130,28864.54,2018-09-07,11,2000,Monsoonal Rain,1
8,2018-08-20,4667,Nigeria,8.23750,10.47480,257477.37,2018-10-02,108,148000,Heavy Rain,1
9,2018-08-22,4670,USA,-155.44100,19.66890,7451.62,2018-08-31,0,50,Tropical Storm Lane,1


In [13]:
outfile = "./data/databases/dartmouth.csv"
data.to_csv(outfile, sep=',')

In [14]:
type(data.index[0])

pandas._libs.tslibs.timestamps.Timestamp

In [15]:
print(list(data.columns.values))

['id_darmouth', 'countries', 'long', 'lat', 'area', 'end_date', 'deaths', 'affected', 'cause', 'in_darmouth']
